In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# The Chess Interface and Game Loop

### Dependencies

In [ ]:
import chess
import chess.svg
import chess.polyglot
import chess.gaviota
import time
import random
import json
from IPython.display import clear_output, SVG, display, HTML
from typing import Callable, Union

import import_ipynb
from Globals import Globals

importing Jupyter notebook from Globals.ipynb


#### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a ValueError is raised and the board state remains unaffected.

###### <b><u>Arguments</u></b>
<b>move</b> <i>(chess.Move)</i> : The move that should be pushed to the move stack.

###### <b><u>Side effects</u></b>
Pushes a move to the board's move stack, assuming the move was valid and legal.

In [ ]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal
del push_legal

---

### Game Class
Lets the user play a game of chess against the AI using a virtual chess board UI. Alternatively, two AIs may play against each other. The game may use any algorithm for the AI(s) to make their moves.
<br><br>

#### \_\_init\_\_
The constructor for the Game class. Gets called whenever a new Game instance is created.
###### __<u>Arguments</u>__
``make_move_algorithm_white (Callable, default: None):``  
The function that the AI controlling the white pieces should use to make a new move, or None if the white pieces are player-controlled.

``make_move_algorithm_black (Callable, default: None):``  
The function that the AI controlling the black pieces should use to make a new move, or None if the black pieces are player-controlled. 

``search_depth_white (int, default: 3):``  
The search depth for the white AI's search algorithm. 

``search_depth_black (int, default: 3):``  
The search depth for the black AI's search algorithm. 

``opening_book (str, default: 'Resources/baron30.bin'):``  
The path to the desired opening book the AIs will use, or None if no opening book is desired.

``endgame_tablebase_dir (str, default: 'Resources/Gaviota'):``  
The path to the directory of the desired endgame tablebase the AIs will use, or None if no endgame tablebase is desired.

###### <b><u>Side effects</u></b>
If neither white nor black is set to be an AI, the game will exit. At least one AI is required to play the game.


In [ ]:
class Game:
    def __init__(
            self,
            make_move_algorithm_white: Callable=None,
            make_move_algorithm_black: Callable=None,
            search_depth_white: int=3,
            search_depth_black: int=3,
            opening_book: str='Resources/baron30.bin',
            endgame_tablebase_dir: str='/datasets/gaviota'
    ):
        self.opening_book = opening_book
        self.endgame_tablebase_dir = endgame_tablebase_dir

        self.test_playthrough = []
        self.problem_playthroughs = {}
        self.wins = []

        self.board = chess.Board()
        self.board.ending = False

        self.move_times = { chess.WHITE: [], chess.BLACK: [] }
        self.make_move = {
            chess.WHITE: make_move_algorithm_white,
            chess.BLACK: make_move_algorithm_black
        }
        self.search_depth = {
            chess.WHITE: search_depth_white,
            chess.BLACK: search_depth_black
        }
        self.is_ai = self.get_ai_state()

        if self.endgame_tablebase_dir is not None:
            self.endgame_tablebase = chess.gaviota.open_tablebase(
                    self.endgame_tablebase_dir)

        # The game requires that at least one player is an AI
        # - it is not a multiplayer chess game
        if not self.is_ai[chess.WHITE] and not self.is_ai[chess.BLACK]:
            raise ValueError('At least one player must be an AI')

#### Game: Class Variables  

``BOARD_DISPLAY_SIZE (int):``  
The size of the displayed chess board in pixels.

``board (chess.Board):``   
The chess board instance that's used for the game.

``make_move (dict<chess.Color, Callable>):``   
The algorithms the AIs, playing white and/or black, use to make a move.

``search_depth (dict<chess.Color, int>):``   
The search depth for the AI's search algorithm for either color.

``is_ai (dict<chess.Color, bool>):``   
Whether a color is AI-controlled (True) or player-controlled (False).

``move_times dict<chess.Color, list>):``   
The time the AI(s) take to make their move.

``test_playthrough (list<tuple<int, chess.Move, bool>>):``  
The progress of the current game playthrough (evaluated move score, move and if the endgame library produced the move), for debugging purposes.

``problem_playthroughs (dict<str, list>):``   
A debug score, move and endgame list (see above) for each test, useful when debugging multiple chess problem tests.

``wins (list<chess.Color>):``   
A list of colors that have won in the previous tests (None in case of a draw).

``opening_book (str):``  
A path to a Polyglot opening book, or None if no opening book is in use.

``endgame_tablebase_dir (str):``  
A path to a Gaviota endgame tablebase directory, or None if no endgame tablebase is in use.

``endgame_tablebase (Union[NativeTablebase, PythonTablebase]):``   
The endgame tablebase object, or None if no endgame tablebase is in use.

In [ ]:
# Constants
Game.BOARD_DISPLAY_SIZE = 500

# Board variables
Game.board = None

# AI and player declarations
Game.make_move = None
Game.search_depth = None
Game.is_ai = None

# Testing declarations
Game.move_times = None
Game.test_playthrough = None
Game.problem_playthroughs = None
Game.wins = None

# Opening book and endgame tablebase declarations
Game.opening_book = None
Game.endgame_tablebase_dir = None
Game.endgame_tablebase = None

#### Game.play  

Starts the game.
Depending on the input arguments there are different game-states and features:  

``(AI, AI):``  
In this case, the chess board is displayed visually while two AIs, specified by the user in the constructor of the Game-class, play against each other. Therefore making moves based on the specified algorithms, depending on the current game- / board-state.  The AIs take turns.
  
  
``(AI, Player) - (Player, AI):``    
In this case, the chess board is displayed visually while an AI, specified by the user in the constructor of the Game-class, and a human-Player compete against each other. Therefore, the AI is making the best move evaluated by the algorithm based on the current game- / board-state. On the other hand, the player can specify the move to make via input (If it's valid and legal).  
The player and AI take turns.

###### <b><u>Arguments</u></b>
``debug (bool, default: False):``  
Indicates, whether the game is started in 'debug-mode' or 'normal-mode'. In debug-mode, further information is tracked. In normal mode, a typical game of chess is started. 

``chess_problem (str, default: None):``  
The name of the chess-problem which should be executed.

``automation (bool, default: True):``  
Whether or not the user has to press a button to get to the next board-state. This should always be true if more than one problem is executed. Furthermore, if automation = True, further information will be tracked.  

``search_depth_auto (bool):``  
Whether or not the max iteration should be evaluated with default max iteration 3, or ideal max iteration defined in Globals.


######  <u>__Side effects__</u> 
- The state of the board in the Game class is changed.  
- The state of is_first_move is changed after the first move.
- The times the AIs take to make a move is tracked and saved.
- The chess board is visually displayed.
- All taken moves are tracked and saved.
- A "Winner-message" is displayed upon the end of the game.


In [ ]:
def play(
    self,
    debug: bool=False,
    chess_problem: str=None,
    search_depth_auto: bool=False,
    automation: bool=True
) -> None:

    use_heuristic = (chess_problem is None)
    if chess_problem is None:
        problem, iterations = None, None
    else:
        problem, iterations = self.set_up_chess_problem(chess_problem)
        
    while not self.board.is_game_over():
        self.display(automation=True)
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()

            # Make move
            if not self.make_opening_move():
                color = self.board.turn
                search_depth = iterations \
                        if search_depth_auto and iterations is not None \
                        else self.search_depth[color]
                score, move, used_endgame = self.make_move[color](
                        self.board, color, search_depth, self.endgame_tablebase,
                        use_heuristic)
                self.test_playthrough.append((score, move.uci(), used_endgame))

            # Add time difference to move times
            time_stamp = time.time() - time_stamp
            self.move_times[not self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        self.board.check_and_set_ending()
        self.display(automation=automation, debug=debug)

    is_draw = self.board.outcome() is None or self.board.outcome().winner is None
    winner = '' if is_draw \
        else 'White' if self.board.outcome().winner == chess.WHITE \
        else "Black"
    winner_message = 'It\'s a draw!' if is_draw else f'{winner} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome()) 
    
    if problem is not None and automation:
        self.set_problem_playthrough(problem)

Game.play = play
del play

#### Game.display
Displays the chess board graphically.

###### __<u>Arguments</u>__

``debug(bool, default: False):``  
Whether or not to wait to clear the output until new output is available to replace it.
   
``automation(bool. default: True): ``  
Whether or not the user has to press a key after each move to continue.  

###### __<u>Side effects</u>__
The current visual output is overwritten.


In [ ]:
def display(self, debug: bool=False, automation: bool=True) -> None:
    clear_output(wait=not debug)
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE,
                lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    display(board_visual)
    if not automation:
        if len(self.board.move_stack) != 0:
            print(self.test_playthrough[-1])
        input('Press any button to continue')

Game.display = display
del display

#### Game.get_check_squares
checks for pieces or squares giving check.
######  <b><u>Returns <i>(chess.Square)</i></u></b>
The square which gives check.
###### __<u>Side effects</u>__
If get_check_squares returns a value, the display function will indicate the square giving check with a red background.
    

In [ ]:
def get_check_square(self) -> chess.Square:
    if self.board.is_check():
        return self.board.checkers().pop()
    return None
            
Game.get_check_square = get_check_square
del get_check_square

#### Game.make_opening_move
Makes a random move from the opening book, if available. If not, no move is made.
###### __<u>Returns _(bool)_</u>__
Whether or not an opening move has been made.

In [ ]:
def make_opening_move(self) -> bool:
    if self.opening_book is None:
        return False

    with chess.polyglot.open_reader(self.opening_book) as reader:
        possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
        if len(possible_moves) > 0:
            move = random.choice(possible_moves)
            self.board.push(move)
            self.test_playthrough.append(('opening library', move.uci(), False))
            return True

    return False

Game.make_opening_move = make_opening_move
del make_opening_move

#### Game.get_ai_state
Defines which players are AIs depending on the make_move parameters in the constructor.

###### __<u>Returns _(dict<chess.Color, bool>)_</u>__
- The color of the player (white or black).
- Whether or not that player is an AI.

In [ ]:
def get_ai_state(self) -> dict:
    return {
        color: self.make_move[color] is not None
            for color in ( chess.WHITE, chess.BLACK )
    }

Game.get_ai_state = get_ai_state
del get_ai_state

#### Game.make_move_human
If the player is human, this method enables the human user to make a move through the input console.
###### __<u>Side effects</u>__
A move is pushed onto the move stack and it is the next player's turn.

In [ ]:
def make_move_human(self) -> None:
    input_prompt = 'Please input your move: '
    while True:
        try:
            move = chess.Move.from_uci(input(input_prompt))
            self.board.push_legal(move)
            return
        except ValueError:
            input_prompt = 'Illegal move, please try again: '
                
Game.make_move_human = make_move_human
del make_move_human

#### Game.save_state
The move stack as well as all the scores are saved in a file, named by the user. The file is stored in the current working directory. 
This file can then later be loaded to open a game.
       
###### __<u>Side effect</u>__  
Execution is paused until a valid file name is given as user input.

In [ ]:
def save_state(self) -> None:
    while True:
        try:
            filename = input('Please input save name:\n') + '.chessgame'
            with open(filename, 'w') as f:
                state = json.dumps(list(self.test_playthrough))
                f.write(state)
            return
        except OSError:
            print(f' {filename} is not a valid filename. Please try again.')
            
Game.save_state = save_state
del save_state

#### Game.load_state
A saved game is loaded. Therefore, the game will be recreated by loading the move stack from a saved game.

###### __<u>Side effect</u>__  
Execution is paused until valid file name is given as user input.

In [ ]:
def load_game(self) -> None:
    while True:
        try:
            filename = input('What is your save file called?:\n')
            if not filename.endswith('.chessgame'):
                filename += '.chessgame'
                
            with open(filename) as f:
                s = f.read()
                d = json.loads(s)
                self.test_playthrough = list(d)
                self.board.clear_stack()
                
                for _, move, _ in self.test_playthrough:
                    self.board.push_legal(chess.Move.from_uci(move))

            return
        except OSError:
            print(f'There is no such file called {filename} - please try again')
    
Game.load_game = load_game
del load_game

#### Game.set_up_chess_problem
Sets the board state to the starting state of a chess problem, using a FEN code. If the given chess problem is invalid, it asks the user for input. Furthermore, it finds the minimum amount of moves to solve the chess problem, which can be used as a search depth to theoretically guarantee that a search algorithm can find the optimal solution.


###### <b><u>Arguments</u></b>
``chess_problem (str):``  
The abbreviated name of the chess problem (see Globals.CHESS_PROBLEMS).

###### <u>__Returns__ _(str, int)_</u>
- The abbreviated name of the problem that is played
- The minimum amount of moves to solve the chess problem.

###### <b><u>Side effects</u></b>
If no chess problem is given as an argument, the user is prompted for input.

In [ ]:
def set_up_chess_problem(self, chess_problem: str) -> (str, int):
    # Clear the test_playthrough list - this dict may have contained moves from
    # the last playthrough
    self.test_playthrough.clear()

    if chess_problem not in Globals.CHESS_PROBLEMS:
        for p in Globals.CHESS_PROBLEMS:
            print(f' {p},')
        problem = input('Please choose a chess problem from the above. \n' \
                + 'You can find further information about the given problems ' \
                + 'in Globals.ipynb\n')
        while problem not in Globals.CHESS_PROBLEMS:
            clear_output()
            for p in Globals.CHESS_PROBLEMS:
                print(f' {p},')
            problem = input('Wrong input, please try again. Type one of the '
                    + 'above abbreviations to initialze the test')
    else:
        problem = chess_problem

    if problem is not None:
        self.board.set_fen(Globals.CHESS_PROBLEMS[problem])

    iterations = Globals.CHESS_PROBLEM_MOVE_COUNT[problem] \
            if problem in Globals.CHESS_PROBLEM_MOVE_COUNT \
            else None
    return problem, iterations

Game.set_up_chess_problem = set_up_chess_problem
del set_up_chess_problem

#### Game.set_problem_playthrough
Registers a playthrough of the current chess problem as defined in test_playthrough. This is added to the problems_playthrough dictionary so that the playthrough is saved even if another test is run. Furthermore, the winning color is appended to a list of winners.

###### <b><u>Arguments</u></b>
``problem (str):``  
The abbreviated name of the chess problem.

###### <b><u>Side effects</u></b>
The playthrough is added to the test_playthrough dictionary and the winner (or draw) is appended to the winners list.

In [ ]:
def set_problem_playthrough(self, problem: str) -> None:
    self.problem_playthroughs[problem] = list(self.test_playthrough)
    winner = None
    if self.board.outcome() is None or self.board.outcome().winner is None:
        winner = None 
    elif self.board.outcome().winner == chess.WHITE:
        winner = 'white'
    else:
        winner = 'black'
            
    self.wins.append(winner)
    
Game.set_problem_playthrough = set_problem_playthrough
del set_problem_playthrough

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>